In [13]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import re
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


pd.set_option("display.max_columns", None)

In [71]:
import logging

logging.basicConfig(
    level=logging.INFO,  # default level shown
    format="%(asctime)s [%(levelname)s] %(message)s"
)

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("/Users/thijsmanuel/Documents/ECON4130_Term_Paper/scrape.log"),
        logging.StreamHandler()  # still show on console
    ]
)


In [99]:
def parse_time(t):
    """Convert HKJC finish time (e.g., 1.22.05) into total seconds."""
    try:
        m, s, h = t.split(".")
        return int(m) * 60 + int(s) + int(h) / 100
    except Exception:
        return None
    
def parse_hkjc_date(x):
    # Try 4-digit year first
    try:
        return pd.to_datetime(x, format="%d/%m/%Y")
    except:
        pass
    
    # Try 2-digit year
    try:
        return pd.to_datetime(x, format="%d/%m/%y")
    except:
        return pd.NaT
    

def scrape_single_race(race_date, racecourse, race_no, driver=None):
    """Scrape one HKJC race and return ML-ready DataFrame."""

    # # --- driver setup ---
    # created_driver = False
    # if driver is None:
    #     options = Options()
    #     options.add_argument("-headless")
    #     driver = webdriver.Firefox(options=options)
    #     created_driver = True

    # Proper date formatting for HKJC URLs
    date_str = pd.to_datetime(race_date).strftime("%Y/%m/%d")
    race_dt = pd.to_datetime(race_date)

    race_url = (
        f"https://racing.hkjc.com/racing/information/english/"
        f"Racing/LocalResults.aspx?RaceDate={date_str}&Racecourse={racecourse}&RaceNo={race_no}"
    )

    driver.get(race_url)
    if "No information." in driver.page_source:
        raise ValueError("Empty race page")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # -------- Extract race header (race_id) ----------
    race_header = soup.find("td", colspan="16")
    race_id_match = re.search(r"\((\d+)\)", race_header.get_text(strip=True)) if race_header else None
    race_id = int(race_id_match.group(1)) if race_id_match else None

    # -------- Extract race distance ----------
    dist_td = soup.find("td", string=re.compile(r"\d+\s*M", re.IGNORECASE))
    dist_match = re.search(r"(\d+)\s*M", dist_td.get_text()) if dist_td else None
    race_distance = int(dist_match.group(1)) if dist_match else None

    # -------- Extract race class ----------
    class_td = soup.find("td", string=re.compile(r"\bClass\s+(\d+)\b", re.IGNORECASE))
    class_match = re.search(r"\bClass\s+(\d+)\b", class_td.get_text(strip=True), re.IGNORECASE) if class_td else None
    race_class = int(class_match.group(1)) if class_match else None

    # -------- Extract race rating ----------
    rating_td = soup.find("td", string=re.compile(r"\(\d+\s*[-–]\s*\d+\)", re.IGNORECASE))
    rating_match = re.search(r"\((\d+)\s*[-–]\s*(\d+)\)", rating_td.get_text(strip=True), re.IGNORECASE) if rating_td else None

    rating_max, rating_min = rating_match.groups()  # first number is the HIGH side in HK racing cards
    rating_min = int(rating_min) if rating_match else None
    rating_max = int(rating_max) if rating_match else None
    rating_min = pd.to_numeric(rating_min, errors='coerce')
    rating_max = pd.to_numeric(rating_max, errors='coerce')

    # -------- Extract surface ----------
    surface_td = soup.find("td", string=re.compile(r"\b(Turf|AWT)\b", re.IGNORECASE))
    surface_match = re.search(r"\b(Turf|AWT)\b", surface_td.get_text(strip=True), re.IGNORECASE) if surface_td else None
    surface = surface_match.group(1) if surface_match else None

    # -------- Get results table body ----------
    tbody = soup.find("tbody", class_="f_fs12")
    if tbody is None:
        return pd.DataFrame()  # nothing to scrape

    # -------- Extract horses (ID + number + finish_place) ----------
    horses = []

    for row in tbody.find_all("tr"):
        cells = row.find_all("td")
        if len(cells) < 3:
            continue

        finish_str = cells[0].get_text(strip=True)
        horse_no = cells[1].get_text(strip=True)
        declared_weight = cells[6].get_text(strip=True)
        jockey = cells[3].get_text(strip=True)
        draw = cells[7].get_text(strip=True)
        odds = cells[-1].get_text(strip=True)

        # some rows might be "SCR", etc. – skip non-numeric finishes
        try:
            finish_place = int(finish_str)
        except ValueError:
            continue

        link = row.find("a", href=re.compile("HorseId"))
        m = re.search(r"HorseId=([A-Z0-9_]+)", link["href"]) if link else None
        if not m:
            continue

        horse_id = m.group(1)
        horses.append({
            "horse_id": horse_id,
            "horse_no": horse_no,
            "finish_place": finish_place,
            "declared_weight": declared_weight,
            "jockey": jockey,
            "draw": draw,
            "odds": odds,
        })

    # -------- Collect final rows for this race ----------
    rows = []

    for horse in horses[:]:
        horse_id = horse["horse_id"]
        horse_no = horse["horse_no"]
        finish_place = horse["finish_place"]
        declared_weight = horse["declared_weight"]
        jockey = horse['jockey']
        draw = horse['draw']
        odds = horse["odds"]

        horse_url = (
            f"https://racing.hkjc.com/racing/information/english/"
            f"Horse/OtherHorse.aspx?HorseId={horse_id}"
        )
        driver.get(horse_url)
        time.sleep(1)  # be polite to HKJC servers
        page = BeautifulSoup(driver.page_source, "html.parser")

        table = page.find("table", class_="bigborder")
        if table is None:
            # no history table? treat as no history
            rows.append({
                "horse_id": horse_id,
                "horse_no": horse_no,
                "finish_place": finish_place,
                "time_since_last_race": None,
                "total_races": 0,
                "avg_time": None,
                "avg_place": None,
                "avg_weight": None,
                "declared_weight": declared_weight,
                "avg_class": None,
                "dist_experience": 0,
                "race_id": race_id,
                "race_date": race_dt,
                "race_distance": race_distance,
                "race_class": race_class,
                "rating_min": rating_min,   
                "rating_max": rating_max,
                "has_history": 0,
                "odds": odds,
            })
            continue

        # Convert history table to DataFrame
        rows_raw = [
            [c.get_text(strip=True) for c in r.find_all(["td", "th"])]
            for r in table.find_all("tr")
        ]
        hist_df = pd.DataFrame(rows_raw).dropna(how="all")
        hist_df.columns = hist_df.iloc[0]
        hist_df = hist_df[1:]

        # Clean historical data
        hist_df["Date"] = hist_df["Date"].apply(parse_hkjc_date) 
        hist_df = hist_df[hist_df["Date"] < race_dt]  # only past races

        # Filter by race distance
        dist_df = hist_df[hist_df["Dist."] == str(race_distance)].copy()
        dist_df["FinishSeconds"] = dist_df["Finish Time"].apply(parse_time)

        dist_df["racecourse"] = dist_df["RC/Track/Course"].apply(
            lambda x: "HV" if "HV" in str(x).upper() else "ST")
        
        dist_df["is_course"] = (dist_df["racecourse"] == racecourse).astype(int)

        dist_df['surface'] = dist_df["RC/Track/Course"].apply(
            lambda x: "TURF" if "TURF" in str(x).upper() else "AWT")
        dist_df['is_turf'] = (dist_df['surface'] == surface).astype(int)

        jockey_df = dist_df[dist_df['Jockey'] == jockey]    
        jockey_win_rate = (jockey_df['Pla.'] == '1').mean() if not jockey_df.empty else None
        jockey_horse_rides = len(jockey_df)
        jockey_horse_avg_place = pd.to_numeric(jockey_df['Pla.'], errors='coerce').mean() if not jockey_df.empty else None

        time_since_last_race = race_dt - hist_df['Date'].max()
        current_rating = hist_df['Rtg.'].iloc[0] if not hist_df.empty else None
        course_pref = dist_df["is_course"].mean() if not dist_df.empty else None

        
        rows.append({
            "horse_id": horse_id,
            "horse_no": horse_no,

            "has_history": int(len(hist_df) > 0),
            "total_races": len(hist_df),
            "time_since_last_race": time_since_last_race.days,
            "avg_time": dist_df["FinishSeconds"].mean(),
            "avg_place": pd.to_numeric(dist_df["Pla."], errors="coerce").mean(),
            "avg_weight": pd.to_numeric(dist_df["Declar.Horse Wt."], errors="coerce").mean(),
            "avg_class": pd.to_numeric(dist_df["RaceClass"], errors="coerce").mean(),
            "dist_experience": len(dist_df),
            "course_pref": course_pref,
            "surface_pref": dist_df["is_turf"].mean() if not dist_df.empty else None,
            'rating': current_rating,  

            "jockey_horse_win_rate": jockey_win_rate,  
            "jockey_horse_rides": jockey_horse_rides,
            "jockey_horse_avg_place": jockey_horse_avg_place,
                
            "race_id": race_id,
            "race_date": pd.to_datetime(race_date),
            "race_distance": race_distance,
            "race_class": race_class,
            "declared_weight": declared_weight,
            "weight_diff": pd.to_numeric(declared_weight, errors="coerce") - pd.to_numeric(dist_df["Declar.Horse Wt."], errors="coerce").mean() if len(dist_df) > 0 else None,
            "rating_min": rating_min,   
            "rating_max": rating_max,
            "draw": draw,

            "finish_place": finish_place,      # <- actual target
            "odds": odds,
        })

    return pd.DataFrame(rows)


In [100]:
options = Options()
options.add_argument("-headless")
options.headless = True
shared_driver = webdriver.Firefox(options=options)

all_races = []

import pandas as pd

all_races = []

start = pd.to_datetime("2023-01-01")   # HK season start
end   = pd.to_datetime("2025-01-01")   # HK season end

dates = pd.date_range(start, end, freq="D")

for date in dates:
    weekday = date.weekday()   # Monday=0 ... Sunday=6
    date_str = date.strftime("%Y-%m-%d")

    # Determine which course to scrape
    if weekday == 2:   # Wednesday
        courses = ["HV"]
    elif weekday == 6: # Sunday
        courses = ["ST"]
    elif weekday == 5: # Saturday
        courses = ["ST"]   # Usually ST
    else:
        continue  # no racing on this day → skip quick

    logger.info(f"Scraping race day {date_str} at {course}")

    for course in courses:
        for race_no in range(1, 13):
            try:
                df = scrape_single_race(date_str, course, race_no, driver=shared_driver)
                if not df.empty:
                    logger.info(f"  Scraped {course} Race {race_no}")
                    all_races.append(df)
            except Exception as e:
                logger.warning(f"Skipped {course} Race {race_no}: {e}")
                continue

final_dataset = pd.concat(all_races, ignore_index=True).round(2)

shared_driver.quit()
print(final_dataset.shape)


2025-12-04 02:10:33,734 [INFO] Scraping race day 2023-01-01 at ST
2025-12-04 02:10:56,598 [INFO]   Scraped ST Race 1
2025-12-04 02:11:18,154 [INFO]   Scraped ST Race 2
2025-12-04 02:11:36,396 [INFO]   Scraped ST Race 3
2025-12-04 02:11:54,077 [INFO]   Scraped ST Race 4
2025-12-04 02:12:16,068 [INFO]   Scraped ST Race 5
2025-12-04 02:12:37,382 [INFO]   Scraped ST Race 6
2025-12-04 02:12:54,154 [INFO]   Scraped ST Race 7
2025-12-04 02:12:55,061 [WARNING] Skipped ST Race 8: 'NoneType' object has no attribute 'groups'
2025-12-04 02:13:12,146 [INFO]   Scraped ST Race 9
2025-12-04 02:13:35,000 [INFO]   Scraped ST Race 10
2025-12-04 02:13:56,498 [INFO]   Scraped ST Race 11
2025-12-04 02:13:57,106 [WARNING] Skipped ST Race 12: 'NoneType' object has no attribute 'groups'
2025-12-04 02:13:57,107 [INFO] Scraping race day 2023-01-04 at ST
2025-12-04 02:14:14,287 [INFO]   Scraped HV Race 1
2025-12-04 02:14:31,886 [INFO]   Scraped HV Race 2
2025-12-04 02:14:50,347 [INFO]   Scraped HV Race 3
2025-12-

(16742, 27)


In [106]:
final_dataset

,has_history,total_races,time_since_last_race,avg_time,avg_place,avg_weight,avg_class,dist_experience,course_pref,surface_pref,rating,jockey_horse_win_rate,jockey_horse_rides,jockey_horse_avg_place,race_distance,race_class,declared_weight,weight_diff,rating_min,rating_max,draw,finish_place,odds
0,1,29,14.0,82.63,4.73,1204.64,4.36,11,1.0,1.0,39,0.0,4,3.75,1400,5,1224,19.36,0,40,2,1,4.8
1,1,7,14.0,82.91,7.00,1178.00,4.00,2,1.0,1.0,42,NaN,0,NaN,1400,5,1175,-3.00,0,40,6,2,4.3
2,1,23,28.0,84.74,11.50,1152.00,4.50,2,1.0,1.0,29,NaN,0,NaN,1400,5,1187,35.00,0,40,4,3,13
3,1,35,11.0,83.02,6.86,992.43,4.86,7,1.0,1.0,30,0.0,1,1.00,1400,5,1015,22.57,0,40,7,4,21
4,1,13,14.0,83.41,7.50,1069.25,4.25,4,1.0,1.0,32,0.0,1,4.00,1400,5,1095,25.75,0,40,1,5,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16737,1,17,11.0,82.34,5.55,1163.00,3.09,11,1.0,1.0,63,NaN,0,NaN,1400,3,1177,14.00,60,80,9,10,41
16738,1,11,32.0,82.97,8.00,1000.00,3.00,1,1.0,1.0,71,0.0,1,8.00,1400,3,995,-5.00,60,80,12,11,22
16739,1,1,7.0,NaN,NaN,NaN,NaN,0,NaN,NaN,68,NaN,0,NaN,1400,3,1132,NaN,60,80,10,12,44
16740,1,10,35.0,82.14,3.33,1039.50,3.67,6,1.0,1.0,72,0.0,6,3.33,1400,3,1050,10.50,60,80,14,13,55


In [102]:
final_dataset.to_excel('/Users/thijsmanuel/Documents/ECON4130_Term_Paper/Data/horse_training_data.xlsx', index=False)

In [107]:
final_dataset.corr()['finish_place'].sort_values()

ValueError: could not convert string to float: '--'

In [17]:
horse_url = 'https://racing.hkjc.com/racing/information/english/Horse/OtherHorse.aspx?HorseId=HK_2022_H436'

# Set up selenium to use Firefox
options = Options()
options.add_argument('-headless') #No need to open a browser window
driver = webdriver.Firefox(options=options)

# Fetch the page
driver.get(horse_url)

# Make a copy of the page source
page_source = driver.page_source

# we can close the browser and clear out Selenium
driver.quit()

In [18]:
horse_soup = BeautifulSoup(page_source,'html.parser')

table = horse_soup.find("table", class_="bigborder")

In [62]:
need = final_dataset[final_dataset['race_id'] == 318][['horse_no',]].copy()

In [63]:
from bs4 import BeautifulSoup
import pandas as pd

url = "https://racing.hkjc.com/racing/information/English/racing/LocalResults.aspx"

# Set up selenium to use Firefox
options = Options()
options.add_argument('-headless') #No need to open a browser window
driver = webdriver.Firefox(options=options)

# Fetch the page
driver.get(url)

# Make a copy of the page source
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

# Locate the Dividend table by a known selector
# fallback to text search if the class changes
table = None
for tbl in soup.find_all("table"):
    if "Dividend" in tbl.text:
        table = tbl
        break

data = []
current_pool = None

for tr in table.find("tbody").find_all("tr"):
    tds = tr.find_all("td")

    if not tds:
        continue

    # If first cell has rowspan — it's a new Pool name
    if tds[0].get("rowspan"):
        current_pool = tds[0].get_text(strip=True)
        win_comb = tds[1].get_text(strip=True)
        dividend = tds[2].get_text(strip=True)
    else:
        # Continue using the previous pool
        win_comb = tds[0].get_text(strip=True)
        dividend = tds[1].get_text(strip=True)

    data.append({
        "Pool": current_pool,
        "Winning Combination": win_comb,
        "Dividend (HK$)": dividend
    })

df_dividends = pd.DataFrame(data)
print(df_dividends)


               Pool        Winning Combination Dividend (HK$)
0               WIN                          2          69.00
1             PLACE                          2          22.50
2             PLACE                          5          19.50
3             PLACE                          4          16.50
4          QUINELLA                        2,5         161.00
5    QUINELLA PLACE                        2,5          61.00
6    QUINELLA PLACE                        2,4          51.00
7    QUINELLA PLACE                        4,5          47.00
8          FORECAST                        2,5         349.00
9            TIERCE                      2,5,4       1,338.00
10             TRIO                      2,4,5         201.00
11          FIRST 4                    2,4,5,6         547.00
12          QUARTET                    2,5,4,6      11,160.00
13       7TH DOUBLE                       10/2       6,710.50
14       7TH DOUBLE                       10/5         995.50
15      

In [67]:
target_pools = ["WIN", "PLACE", "QUINELLA", "QUINELLA PLACE", "FORECAST"]
df_div = df_dividends[df_dividends["Pool"].isin(target_pools)].copy()

df_single = df_div[df_div["Winning Combination"].str.contains(",") == False].copy()
df_single["horse_no"] = df_single["Winning Combination"].astype(int)

df_pivot = df_single.pivot(index="horse_no", columns="Pool", values="Dividend (HK$)").fillna(0)
df_pivot.reset_index(inplace=True)

need["horse_no"] = need["horse_no"].astype(int)


df_result = need.merge(df_pivot, on="horse_no", how="left").fillna(0)


In [68]:
df_result

,horse_no,PLACE,WIN
0,1,0,0
1,7,0,0
2,3,0,0
3,8,0,0
4,11,0,0
5,4,16.50,0
6,6,0,0
7,12,0,0
8,10,0,0
9,5,19.50,0


In [69]:
df_div

,Pool,Winning Combination,Dividend (HK$)
0,WIN,2,69.00
1,PLACE,2,22.50
2,PLACE,5,19.50
3,PLACE,4,16.50
4,QUINELLA,"2,5",161.00
5,QUINELLA PLACE,"2,5",61.00
6,QUINELLA PLACE,"2,4",51.00
7,QUINELLA PLACE,"4,5",47.00
8,FORECAST,"2,5",349.00
